# Section 0: Preliminaries
Please execute the following code blocks to setup the visualization, relevant imports, some helper functions and the robot selector.

In [2]:
%%bash --bg
rviz -d ~/giskard_examples/launch/rviz_config/standalone.rviz

In [23]:
# imports and helper functions
import rospy
from geometry_msgs.msg import PoseStamped, Point, Quaternion, Vector3Stamped, PointStamped, Vector3
from tf.transformations import quaternion_from_matrix, quaternion_about_axis
from giskardpy.goals.weight_scaling_goals import MaxManipulabilityLinWeight, BaseArmWeightScaling
from copy import deepcopy
from giskard_msgs.msg import WorldBody, CollisionEntry, WorldGoal, GiskardError
import numpy as np
from giskardpy.goals.joint_goals import JointPositionList
from giskardpy.motion_graph.monitors.joint_monitors import JointGoalReached
from IPython.display import display, Image
from pdf2image import convert_from_path
import os
import glob
import ipywidgets as widgets
from IPython.display import display
import subprocess


# Define some helper functions
def reset_giskard():
    giskard.clear_motion_goals_and_monitors()
    giskard.world.clear()
    if ROBOT == 'stretch':
        default_pose = {
            'joint_gripper_finger_left': 0.6,
            'joint_gripper_finger_right': 0.6,
            'joint_right_wheel': 0.0,
            'joint_left_wheel': 0.0,
            'joint_lift': 0.5,
            'joint_arm_l3': 0.05,
            'joint_arm_l2': 0.05,
            'joint_arm_l1': 0.05,
            'joint_arm_l0': 0.05,
            'joint_wrist_yaw': 0.0,
            'joint_head_pan': 0.0,
            'joint_head_tilt': 0.0
        }
    elif ROBOT == 'pr2':
        default_pose = {
            'r_elbow_flex_joint': -0.15,
            'r_forearm_roll_joint': 0,
            'r_shoulder_lift_joint': 0,
            'r_shoulder_pan_joint': 0,
            'r_upper_arm_roll_joint': 0,
            'r_wrist_flex_joint': -0.10001,
            'r_wrist_roll_joint': 0,
            'l_elbow_flex_joint': -0.15,
            'l_forearm_roll_joint': 0,
            'l_shoulder_lift_joint': 0,
            'l_shoulder_pan_joint': 0,
            'l_upper_arm_roll_joint': 0,
            'l_wrist_flex_joint': -0.10001,
            'l_wrist_roll_joint': 0,
            'torso_lift_joint': 0.2,
            'head_pan_joint': 0,
            'head_tilt_joint': 0,
            'l_gripper_l_finger_joint': 0.55,
            'r_gripper_l_finger_joint': 0.55
        }
    done = giskard.monitors.add_set_seed_configuration(default_pose)
    base_pose = PoseStamped()
    base_pose.header.frame_id = 'map'
    base_pose.pose.position = Point(0, 0, 0)
    base_pose.pose.orientation.w = 1
    done2 = giskard.monitors.add_set_seed_odometry(base_pose=base_pose)
    giskard.motion_goals.allow_all_collisions()
    giskard.monitors.add_end_motion(start_condition=f'{done} and {done2}')
    giskard.execute()
    giskard.clear_motion_goals_and_monitors()


def visualize_last_task_graph():
    folder_path = '../ROS_WS/src/giskardpy/tmp/task_graphs'
    files = glob.glob((os.path.join(folder_path, '*')))
    latest_file = max(files, key=os.path.getmtime)
    print(latest_file)
    display(Image(filename=latest_file))


def visualize_last_gant_diagram():
    folder_path = '../ROS_WS/src/giskardpy/tmp/gantt_charts'
    files = glob.glob((os.path.join(folder_path, '*')))
    latest_file = max(files, key=os.path.getmtime)
    images = convert_from_path(latest_file)
    print(latest_file)
    display(images[0])


# global variables
ROBOT = 'pr2'
single_joint_goal = {'torso_lift_joint': 0.3}
tool_frame = 'l_gripper_tool_frame'
cam_frame = 'wide_stereo_optical_frame'
base_link = 'base_footprint'
gripper_joint = 'l_gripper_l_finger_joint'
gripper_joint_open = 0.55
gripper_joint_close = 0.1

# List of available launch files
launch_files = {
    'PR2': 'giskardpy_pr2_standalone_vrb.launch',
    'Stretch': 'giskardpy_stretch_standalone_vrb.launch'
}

# Dropdown widget
dropdown = widgets.Dropdown(
    options=launch_files.keys(),
    value='PR2',
    description='Select Robot:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='200px')
)

# Button widget
button = widgets.Button(
    description='Start Launch File',
    button_style='success',
)


def update_global_variables(robot):
    global ROBOT
    global single_joint_goal
    global tool_frame
    global cam_frame
    global base_link
    global gripper_joint
    global gripper_joint_open
    global gripper_joint_close
    ROBOT = robot
    if ROBOT == 'stretch':
        single_joint_goal = {'joint_lift': 0.7}
        tool_frame = 'link_grasp_center'
        cam_frame = 'camera_color_optical_frame'
        base_link = 'base_link'
        gripper_joint = 'joint_gripper_finger_left'
        gripper_joint_open = 0.55
        gripper_joint_close = 0.1
    elif ROBOT == 'pr2':
        single_joint_goal = {'torso_lift_joint': 0.3}
        tool_frame = 'l_gripper_tool_frame'
        cam_frame = 'wide_stereo_optical_frame'
        base_link = 'base_footprint'
        gripper_joint = 'l_gripper_l_finger_joint'
        gripper_joint_open = 0.55
        gripper_joint_close = 0.1


# Function to start the selected ROS launch file
def on_button_click(b):
    selected_launch_file = launch_files[dropdown.value]

    update_global_variables(dropdown.value.lower())

    try:
        result = subprocess.run('rosnode kill /giskard', shell=True, check=True, stdout=subprocess.PIPE,
                                stderr=subprocess.PIPE)
        print(result.stdout.decode())
        print(result.stderr.decode())
    except subprocess.CalledProcessError as e:
        print(f"Error occurred: {e.stderr.decode()}")

    print(f"Starting ROS launch file: {selected_launch_file}")
    command = f"roslaunch giskardpy {selected_launch_file}"
    LAUNCH_PROCESS = subprocess.Popen(['/bin/bash', '-c', command],
        stdout=subprocess.DEVNULL,
        stderr=subprocess.DEVNULL, shell=False)


# Attach the event handler to the button
button.on_click(on_button_click)

# Display widgets
display(dropdown)
display(button)
rospy.init_node('giskard_examples')

Dropdown(description='Select Robot:', layout=Layout(width='200px'), options=('PR2', 'Stretch'), style=Descript…

Button(button_style='success', description='Start Launch File', style=ButtonStyle())

After you click the button, please wait until you see the robot in the visualization window before clicking again.

Right now you can select between the PR2 and the Stretch robot, which have fundamentally different kinematics. You can see that directly when comparing their visualizations. A less obvious difference is that the Stretch uses a differential drive base, while the PR2 uses an omnidirectional drive base. Therefore, throughout the tutorials you will see that to move their bases different code might be used.

# Section 1: Introduction
Giskard is a constraint- and optimization-based whole-body motion planning/control framework.
In a nutshell Giskard constructs and solves the following quadratic problem
\begin{equation}
  \begin{aligned}
  \min_{\mathbf{s}} \quad & \mathbf{s}^T\mathbf{Hs} + \mathbf{C}^T \mathbf{s}\\
  \textrm{s.t.} \quad & \mathbf{l}_A < \mathbf{As} < \mathbf{u}_A\\
    &\mathbf{l} < \mathbf{s} < \mathbf{u}    \\
  \end{aligned}
\end{equation}

Where $\mathbf{A}$ contains the Jacobian of all task functions; $\mathbf{l}_A$ and $\mathbf{u}_A$ contain lower and upper limits for the Jacobian; $\mathbf{s} = (\dot{\mathbf{q}}, \mathbf{c})^T$ is the control vector consisting of joint velocities $\dot{\mathbf{q}}$ and slack variables $\mathbf{c}$ for the constraints; $\mathbf{l}$ and $\mathbf{u}$ contain lower and upper limits for the values in $\mathbf{s}$; $\mathbf{C}$ constructs a linear weight term that is used for manipulability optimization; and $\mathbf{H}$ contains weights for the joint velocities and task functions.
A task function depends on the current joint positions and expresses some value whose derivation should be constrained. Most often, the forward kinematics function is used as a task function to constrain the movement of a specific link.



When using the python wrapper you do not specify constraints directly but rather a Motion Goal to parameterize a predefined set of constraints.
Additionally, the python interface also allows to specify monitors that monitor a mathematical expression against a threshold and evaluate into a binary value.
They are used to start, stop or interrupt motion goals. More complex motion goals might specify their own monitors to chain constraints together.


**Tutorial 1.1: Now initialize the Python Interface. It will connect to the running Giskard instance of your selected robot.**

In [24]:
from giskardpy.python_interface.python_interface import GiskardWrapper

giskard = GiskardWrapper()

**Tutorial 1.2: Use the defined motion goal to move a single joint of the robot**

The simplest possible motion goal call follows the schema:

   1. Define a motion goal.
   2. Define a monitor that checks if the goal was reached.
   3. Define an end motion monitor, which has as start_condition the goal reached monitor.

In [ ]:
reset_giskard()

joint_goal = single_joint_goal
print(joint_goal)
joint_monitor = giskard.monitors.add_joint_position(goal_state=joint_goal)
giskard.motion_goals.add_joint_position(goal_state=joint_goal)
giskard.monitors.add_end_motion(start_condition=joint_monitor)
assert giskard.execute().error.code == GiskardError.SUCCESS

**Tutorial 1.3: Use the defined cartesian pose goal to move the tool frame of the robot**

A commonly used goal is a cartesian pose goal. It is used to move a link of the robot to a specific pose. Frequently used links are the endeffector links or the base link.

**Exercise: Change the orientation and position of the goal_pose and explore the generated movements**

In [7]:
reset_giskard()

# Define a goal pose in the map coordinate system
goal_pose = PoseStamped()
goal_pose.header.frame_id = 'map'
goal_pose.pose.orientation = Quaternion(*quaternion_from_matrix([[1, 0, 0, 0],
                                                                 [0, 1, 0, 0],
                                                                 [0, 0, 1, 0],
                                                                 [0, 0, 0, 1]]))
goal_pose.pose.position.x = 1.2
goal_pose.pose.position.y = -0.2
goal_pose.pose.position.z = 0.7
# specify the cartesian pose goal to move 'l_gripper_tool_frame' towards goal_pose by using the kinematic chain between 'l_gripper_tool_frame' and 'map'
giskard.motion_goals.add_cartesian_pose(goal_pose=goal_pose, root_link='map', tip_link=tool_frame)
# specify a monitor that is active when it is below the given thresholds and use it to end the motion
pose_monitor = giskard.monitors.add_cartesian_pose(goal_pose=goal_pose, root_link='map', tip_link=tool_frame,
                                                   position_threshold=0.01,
                                                   orientation_threshold=0.01)
# add a monitor that automatically stops the motion when the trajectory is too long
giskard.monitors.add_max_trajectory_length(max_trajectory_length=30)
# use the pose monitor to end the motion when it is true
giskard.monitors.add_end_motion(start_condition=pose_monitor)
# in the case that there is no specific end motion condition the default end motion condition stops the motion when the robot stops moving i.e. the QP is in a local minimum.
# giskard.add_default_end_motion_conditions()
assert giskard.execute().error.code == GiskardError.SUCCESS

**Tutorial 1.4: Add a box to the world and specify a cartesian pose goal in its coordinate frame**

It is possible to add objects in the form of primitive shapes, meshes or URDF's to the world representation of Giskard.
Afterward, goal poses can be specified in the coordinate frames of the new entities.

**Exercise: Change the box_pose and observe how giskard still reaches the same goal pose relative to the box**

In [ ]:
reset_giskard()

# adding a box to the world
box_pose = PoseStamped()
box_pose.header.frame_id = 'map'
box_pose.pose.orientation.w = 1
box_pose.pose.position.x = 1.2
box_pose.pose.position.y = -0.2
box_pose.pose.position.z = 0.7
giskard.world.add_box(name='mybox', size=(0.1, 0.05, 0.2), pose=box_pose, parent_link='map')

# specfy a goal pose 8cm in front of the box origin frame
goal_pose = PoseStamped()
goal_pose.header.frame_id = 'mybox'
goal_pose.pose.orientation.w = 1
goal_pose.pose.position.x = -0.08
goal_pose.pose.position.y = 0
goal_pose.pose.position.z = 0
giskard.motion_goals.add_cartesian_pose(goal_pose=goal_pose, root_link='map', tip_link=tool_frame)
pose_monitor = giskard.monitors.add_cartesian_pose(goal_pose=goal_pose, root_link='map', tip_link=tool_frame,
                                                   position_threshold=0.01,
                                                   orientation_threshold=0.01)
giskard.monitors.add_end_motion(start_condition=pose_monitor)
giskard.monitors.add_max_trajectory_length(max_trajectory_length=30)
assert giskard.execute().error.code == GiskardError.SUCCESS

**Tutorial 1.5: Use sets of atomic constraints to specify a motion instead of using a full 6D-Pose**

When using the cartesian pose goals all 6 degrees of freedom of the end effector link are fully constrained. One Advantage of constraint-based control is that one doesn't have to constrain all degrees of freedom.
The unconstrained DOF can then be exploited during the motion, for example in the collision avoidance or other secondary tasks.
To use this in the python interface one can use motion goals that constrain only a singular value, sometimes called constraints on feature functions or atomic constraints.
For a feature function constraints have to defined two features, a controlled feature (Point or Vector) related to the robot, and reference feature which is the target/goal.
The name of the function then defines how they will be combined and 0 or 2 constraint values can be provided.
Right now we support the following atomic constraints:

    - Perpendicular(reference_vector, controlled_vector)
    - Height(reference_point, controlled_point, lower_limit, upper_limit)
    - Distance(reference_point, controlled_point, lower_limit, upper_limit)
    - Pointing(reference_point, controlled_vector)
    - Align(reference_vector, controlled_vector)
    - Angle(reference_vector, controlled_vector, lower_limit, upper_limit)

The controlled and reference features will be visualized as points or vectors in space.

Here a distance constraint between the origin points of the tool link and the map coordinate frames is defined.

In [9]:
reset_giskard()
# Define a world feature as a point at the origin of map
world_feature = PointStamped()
world_feature.header.frame_id = 'map'

# Define a robot feature as a point at the origin of the right gripper tool frame
robot_feature = PointStamped()
robot_feature.header.frame_id = tool_frame

# use the distance feature function to constrain the distance between the features to be between 2m and 2.5m
giskard.motion_goals.add_distance(root_link='map',
                                  tip_link=tool_frame,
                                  reference_point=world_feature,
                                  tip_point=robot_feature,
                                  lower_limit=2,
                                  upper_limit=2.5)

giskard.add_default_end_motion_conditions()  # stop when a local minimum was reached
assert giskard.execute().error.code == GiskardError.SUCCESS

**Tutorial 1.6: Combine atomic constraints for grasping**

We can exploit the rotational symmetry of a cylinder for grasping it if we specify the grasping motion with atomic constraints, which allows for free rotation around the cylinder.
Here we use three different atomic constraints to achieve this.

1. Pointing the gripper towards the object center
2. keeping a suitable distance
3. constrain the height to be within the dimensions of the objects

**Exercise a): Change the position of the cylinder and the constraint values, and observe the resulting grasp poses.**

**Exercise b): You might notice that at some positions the rotation of the gripper is not correct. This is due to a missing atomic constraint that allows the gripper_frame to freely rotate around its x-axis. Try to constrain this using the AlignFeatureFunction. You can visualize tf in rviz to see all coordinate frames**

In [ ]:
reset_giskard()

# variables
gripper_link = tool_frame

# adding the object
cyl_pose = PoseStamped()
cyl_pose.header.frame_id = 'map'
cyl_pose.pose.orientation.w = 1
cyl_pose.pose.position = Point(1.2, -0.2, 1.2)
giskard.world.add_cylinder(name='mycyl', height=0.2, radius=0.02, pose=cyl_pose, parent_link='map')

# define the robot features
robot_pointing_feature = Vector3Stamped()
robot_pointing_feature.header.frame_id = gripper_link
robot_pointing_feature.vector = Vector3(1, 0, 0)

robot_point_feature = PointStamped()
robot_point_feature.header.frame_id = gripper_link
robot_point_feature.point = Point(0, 0, 0)

# define the world features
world_cylinder_center_feature = PointStamped()
world_cylinder_center_feature.header.frame_id = 'mycyl'
world_cylinder_center_feature.point = Point(0, 0, 0)

# define the constraints
giskard.motion_goals.add_distance(root_link='map',
                                  tip_link=gripper_link,
                                  reference_point=world_cylinder_center_feature,
                                  tip_point=robot_point_feature,
                                  lower_limit=0.03,
                                  upper_limit=0.03)
giskard.motion_goals.add_height(root_link='map',
                                tip_link=gripper_link,
                                reference_point=world_cylinder_center_feature,
                                tip_point=robot_point_feature,
                                lower_limit=-0.05,
                                upper_limit=0.05)
giskard.motion_goals.add_pointing(root_link='map',
                                  tip_link=gripper_link,
                                  goal_point=world_cylinder_center_feature,
                                  pointing_axis=robot_pointing_feature)

#Exercise 1.4: You can define your features and your motion goal here

giskard.add_default_end_motion_conditions()
assert giskard.execute().error.code == GiskardError.SUCCESS

**Tutorial 1.7: Adding collision avoidance**

Another advantage of having unconstrained DOF in the motion specification can be seen if we put an obstacle in front of the object we want to grasp and configure the collision avoidance to avoid this obstacle with all parts of the robot.
Thanks to the unconstrained DOF, the collision avoidance can guide the gripper automatically to a suitable pose.

**Exercise: Change the position and the size of the obstacle and observe the resulting motions**

PS: this cell includes the solution to the previous exercise

In [11]:
reset_giskard()

# variables
gripper_link = tool_frame

# adding the object
cyl_pose = PoseStamped()
cyl_pose.header.frame_id = 'map'
cyl_pose.pose.orientation.w = 1
cyl_pose.pose.position = Point(1.2, -0.2, 1.2)
giskard.world.add_cylinder(name='mycyl', height=0.2, radius=0.02, pose=cyl_pose, parent_link='map')

obstacle_pose = PoseStamped()
obstacle_pose.header.frame_id = 'mycyl'
obstacle_pose.pose.orientation.w = 1
obstacle_pose.pose.position = Point(-0.1, 0.1, 0)
giskard.world.add_box(name='obstacle', size=(0.02, 0.06, 0.05), pose=obstacle_pose, parent_link='mycyl')

# define the robot features
robot_pointing_feature = Vector3Stamped()
robot_pointing_feature.header.frame_id = gripper_link
robot_pointing_feature.vector = Vector3(1, 0, 0)

robot_point_feature = PointStamped()
robot_point_feature.header.frame_id = gripper_link
robot_point_feature.point = Point(0, 0, 0)

robot_gripper_z_axis_feature = Vector3Stamped()
robot_gripper_z_axis_feature.header.frame_id = gripper_link
robot_gripper_z_axis_feature.vector = Vector3(0, 0, 1)

# define the world features
world_cylinder_center_feature = PointStamped()
world_cylinder_center_feature.header.frame_id = 'mycyl'
world_cylinder_center_feature.point = Point(0, 0, 0)

world_cyl_z_axis_feature = Vector3Stamped()
world_cyl_z_axis_feature.header.frame_id = 'mycyl'
world_cyl_z_axis_feature.vector = Vector3(0, 0, 1)

# define the constraints
giskard.motion_goals.add_distance(root_link='map',
                                  tip_link=gripper_link,
                                  reference_point=world_cylinder_center_feature,
                                  tip_point=robot_point_feature,
                                  lower_limit=0.03,
                                  upper_limit=0.03)
giskard.motion_goals.add_height(root_link='map',
                                tip_link=gripper_link,
                                reference_point=world_cylinder_center_feature,
                                tip_point=robot_point_feature,
                                lower_limit=-0.05,
                                upper_limit=0.05)
giskard.motion_goals.add_pointing(root_link='map',
                                  tip_link=gripper_link,
                                  goal_point=world_cylinder_center_feature,
                                  pointing_axis=robot_pointing_feature)
giskard.motion_goals.avoid_collision(min_distance=0.1, group1='obstacle')
giskard.motion_goals.add_align_planes(root_link='map',
                                      tip_link=gripper_link,
                                      goal_normal=world_cyl_z_axis_feature,
                                      tip_normal=robot_gripper_z_axis_feature)
giskard.add_default_end_motion_conditions()
assert giskard.execute().error.code == GiskardError.SUCCESS

**Tutorial 1.8: Use monitors to evaluate the success of your specified motion**

For each constraint a related monitor can be added to observe their progression and collect feedback in the case of failure where one or many constraints cannot be satisfied.
A motion was not successful if the end motion monitor did not activate and the goal was canceled by another monitor.
In that case the return value of the execute() call can be passed to the get_end_motion_reason() function to obtain a summary of all monitors that are connected to the end motion and are not active.

**Exercise: The output tells you which monitor of the end motion was not successful. Try changing/removing/adding constraint and monitors to realize a successful execution.**

In [ ]:
reset_giskard()

# variables
gripper_link = tool_frame

# adding the object
cyl_pose = PoseStamped()
cyl_pose.header.frame_id = 'map'
cyl_pose.pose.orientation.w = 1
cyl_pose.pose.position = Point(1.2, -0.2, 1.2)
giskard.world.add_cylinder(name='mycyl', height=0.2, radius=0.02, pose=cyl_pose, parent_link='map')

obstacle_pose = PoseStamped()
obstacle_pose.header.frame_id = 'mycyl'
obstacle_pose.pose.orientation.w = 1
obstacle_pose.pose.position = Point(-0.1, 0.1, 0)
giskard.world.add_box(name='obstacle', size=(0.02, 0.06, 0.05), pose=obstacle_pose, parent_link='mycyl')

# define the robot features
robot_pointing_feature = Vector3Stamped()
robot_pointing_feature.header.frame_id = gripper_link
robot_pointing_feature.vector = Vector3(1, 0, 0)

robot_point_feature = PointStamped()
robot_point_feature.header.frame_id = gripper_link
robot_point_feature.point = Point(0, 0, 0)

robot_gripper_z_axis_feature = Vector3Stamped()
robot_gripper_z_axis_feature.header.frame_id = gripper_link
robot_gripper_z_axis_feature.vector = Vector3(0, 0, 1)

# define the world features
world_cylinder_center_feature = PointStamped()
world_cylinder_center_feature.header.frame_id = 'mycyl'
world_cylinder_center_feature.point = Point(0, 0, 0)

world_cyl_z_axis_feature = Vector3Stamped()
world_cyl_z_axis_feature.header.frame_id = 'mycyl'
world_cyl_z_axis_feature.vector = Vector3(0, 0, 1)

# define the constraints
giskard.motion_goals.add_distance(root_link='map',
                                  tip_link=gripper_link,
                                  reference_point=world_cylinder_center_feature,
                                  tip_point=robot_point_feature,
                                  lower_limit=0.0,
                                  upper_limit=0.0)
mon_dist = giskard.monitors.add_distance(root_link='map',
                                         tip_link=gripper_link,
                                         reference_point=world_cylinder_center_feature,
                                         tip_point=robot_point_feature,
                                         lower_limit=-0.001,
                                         upper_limit=0.001)

giskard.motion_goals.add_height(root_link='map',
                                tip_link=gripper_link,
                                reference_point=world_cylinder_center_feature,
                                tip_point=robot_point_feature,
                                lower_limit=-0.05,
                                upper_limit=0.05)
mon_height = giskard.monitors.add_height(root_link='map',
                                         tip_link=gripper_link,
                                         reference_point=world_cylinder_center_feature,
                                         tip_point=robot_point_feature,
                                         lower_limit=-0.05,
                                         upper_limit=0.05)

giskard.motion_goals.add_pointing(root_link='map',
                                  tip_link=gripper_link,
                                  goal_point=world_cylinder_center_feature,
                                  pointing_axis=robot_pointing_feature)
mon_pointing = giskard.monitors.add_pointing_at(root_link='map',
                                                tip_link=gripper_link,
                                                goal_point=world_cylinder_center_feature,
                                                pointing_axis=robot_pointing_feature)

giskard.motion_goals.add_align_planes(root_link='map',
                                      tip_link=gripper_link,
                                      goal_normal=world_cyl_z_axis_feature,
                                      tip_normal=robot_gripper_z_axis_feature)
mon_align = giskard.monitors.add_vectors_aligned(root_link='map',
                                                 tip_link=gripper_link,
                                                 goal_normal=world_cyl_z_axis_feature,
                                                 tip_normal=robot_gripper_z_axis_feature)

giskard.motion_goals.allow_all_collisions()
giskard.motion_goals.avoid_collision(min_distance=0.08, group1='obstacle')
giskard.monitors.add_end_motion(start_condition=f'{mon_dist} and {mon_height} and {mon_align} and {mon_pointing}')
giskard.monitors.add_cancel_motion(giskard.monitors.add_local_minimum_reached(),
                                   error_message='local minimum reached while monitors are not satisfied')
result = giskard.execute()
if result.error.code != GiskardError.SUCCESS:
    print(giskard.get_end_motion_reason(move_result=result, show_all=False))

**Tutorial 1.9: Beyond constraints. Adapting the cost function to change behaviour.**

The cost function of Giskard consists of a quadratic and a linear term.
The quadratic term weighs all control and constraint slack variables against each other using a weight matrix.
Normally all control variables corresponding to joints have the same weight, which is lower than the constraint slack variables.
This will make Giskard prefer to use joints to satisfy constraints instead of the slack variables, resulting in movement, but will not prioritize any joint over another.
However, redundant robots like the PR2, especially when the base is optimized at the same time, have a lot of nullspace that can be used for better motions though.

For example, we can introduce a using the BaseArmWeightScaling Goal.
It allows us to define joint groups (base joints and arm joints) and change their weights depending on the Euclidean distance between the defined tip link and its goal position.
Now the base joints will be preferred to satisfy all other goals/constraints when the distance is larger.
When the distance becomes smaller, the preference shifts from the base joints to the arm joints.
But there is no hard cut, meaning the base joints can also be used when distance is small if it is needed.

The nullspace can also be used to optimize the manipulability.
The MaxManipulabilityLinWeight Goal adds a linear weight term that maximizes the manipulability of the kinematic chain which is defined by the root link and tip link parameters.
Especially when close to the main goal, this one will move the robot into a less "awkward" pose.

In this tutorial we use the presented goals to show a combination of constraints and cost function augmentations to:
- place both grippers at specific poses (2 6D Pose Constraints)
- keep the line of sight on the left gripper (Pointing Constraint)
- prefer using base movements if the grippers are further away from their goal poses and avoid base movements if they are close to their goals (Quadratic Weight Scaling)
- maximize the manipulability in each arm (Linear weight term in the cost function)

**Exercise: Comment out the goals for the cost function augmentation to compare the crated motions with and without**

In [13]:
reset_giskard()
# This is eventually not feasible for the stretch
if ROBOT == 'pr2':
    js = {
        # 'torso_lift_joint': 0.2999225173357618,
        'head_pan_joint': 0.041880780651479044,
        'head_tilt_joint': -0.37,
        'r_upper_arm_roll_joint': -0.9487714747527726,
        'r_shoulder_pan_joint': -1.0047307505973626,
        'r_shoulder_lift_joint': 0.48736790658811985,
        'r_forearm_roll_joint': -14.895833882874182,
        'r_elbow_flex_joint': -1.392377908925028,
        'r_wrist_flex_joint': -0.4548695149411013,
        'r_wrist_roll_joint': 0.11426798984097819,
        'l_upper_arm_roll_joint': 1.7383062350263658,
        'l_shoulder_pan_joint': 1.8799810286792007,
        'l_shoulder_lift_joint': 0.011627231224188975,
        'l_forearm_roll_joint': 312.67276414458695,
        'l_elbow_flex_joint': -2.0300928925694675,
        'l_wrist_flex_joint': -0.10014623223021513,
        'l_wrist_roll_joint': -6.062015047706399,
    }
    giskard.motion_goals.add_joint_position(goal_state=js)
    giskard.motion_goals.allow_all_collisions()
    giskard.add_default_end_motion_conditions()
    giskard.execute()

goal_pose = PoseStamped()
goal_pose.header.frame_id = 'map'
goal_pose.pose.orientation = Quaternion(*quaternion_from_matrix([[1, 0, 0, 0],
                                                                 [0, 1, 0, 0],
                                                                 [0, 0, 1, 0],
                                                                 [0, 0, 0, 1]]))
goal_pose.pose.position.x = 2.01
goal_pose.pose.position.y = -0.2
goal_pose.pose.position.z = 0.7

goal_pose2 = deepcopy(goal_pose)
goal_pose2.pose.position.y = -0.6
goal_pose2.pose.position.z = 0.8
goal_pose2.pose.orientation = Quaternion(*quaternion_from_matrix([[0, 0, 1, 0],
                                                                  [0, 1, 0, 0],
                                                                  [-1, 0, 0, 0],
                                                                  [0, 0, 0, 1]]))

giskard.motion_goals.add_cartesian_pose(goal_pose=goal_pose, tip_link=tool_frame, root_link='map')
if ROBOT == 'pr2':
    giskard.motion_goals.add_cartesian_pose(goal_pose=goal_pose2, tip_link='r_gripper_tool_frame', root_link='map',
                                            name='cart2')

goal_point = PointStamped()
goal_point.header.frame_id = goal_pose.header.frame_id
goal_point.point = goal_pose.pose.position
pointing_axis = Vector3Stamped()
pointing_axis.header.frame_id = cam_frame
pointing_axis.vector.z = 1
giskard.motion_goals.add_pointing(goal_point=goal_point, tip_link=cam_frame, pointing_axis=pointing_axis,
                                  root_link='map')

x_base = Vector3Stamped()
x_base.header.frame_id = 'base_link'
x_base.vector.x = 1
x_goal = Vector3Stamped()
x_goal.header.frame_id = 'map'
x_goal.vector.x = 1
giskard.motion_goals.add_align_planes(tip_link='base_link',
                                      root_link='map',
                                      tip_normal=x_base,
                                      goal_normal=x_goal)

if ROBOT == 'pr2':
    arm_joints = [
        'torso_lift_joint',
        # 'head_pan_joint',
        # 'head_tilt_joint',
        'r_upper_arm_roll_joint',
        'r_shoulder_pan_joint',
        'r_shoulder_lift_joint',
        'r_forearm_roll_joint',
        'r_elbow_flex_joint',
        'r_wrist_flex_joint',
        'r_wrist_roll_joint',
        'l_upper_arm_roll_joint',
        'l_shoulder_pan_joint',
        'l_shoulder_lift_joint',
        'l_forearm_roll_joint',
        'l_elbow_flex_joint',
        'l_wrist_flex_joint',
        'l_wrist_roll_joint']
    gain = 100000
elif ROBOT == 'stretch':
    arm_joints = ['joint_arm_l3',
                  'joint_arm_l2',
                  'joint_arm_l1',
                  'joint_arm_l0',
                  'joint_wrist_yaw',
                  'joint_wrist_pitch',
                  'joint_wrist_roll']
    gain = 100000

tip_goal = PointStamped()
tip_goal.header.frame_id = 'map'
tip_goal.point = goal_pose.pose.position
giskard.motion_goals.add_motion_goal(motion_goal_class=BaseArmWeightScaling.__name__,
                                     root_link='map',
                                     tip_link=tool_frame,
                                     tip_goal=tip_goal,
                                     gain=gain,
                                     arm_joints=arm_joints,
                                     base_joints=['brumbrum'])
if ROBOT == 'pr2':
    giskard.motion_goals.add_motion_goal(motion_goal_class=MaxManipulabilityLinWeight.__name__,
                                         root_link='torso_lift_link',
                                         tip_link='r_gripper_tool_frame')
    giskard.motion_goals.add_motion_goal(motion_goal_class=MaxManipulabilityLinWeight.__name__,
                                         root_link='torso_lift_link',
                                         tip_link='l_gripper_tool_frame',
                                         name='MaxMal2')
giskard.add_default_end_motion_conditions()
giskard.motion_goals.allow_all_collisions()
assert giskard.execute().error.code == GiskardError.SUCCESS

# Section 2: Environment Manipulation
**Tutorial 2.1: Opening a dishwasher**

If an environment is modelled as articulated links in a URDF, Giskard can load it and control the joint states of the environment when it is connected to links of the robot.
It is of course impossible to control such joints, but assuming that we can, can simplify the definition of environment manipulation goals.
In this example we had already loaded a URDF description of the iai_kitchen onto the parameter server and load it into Giskard's world model.
Then we grasp the dishwasher handle and open the dishwasher using Giskard's open_container goal.
This goal adds constraints to keep the robot gripper at the same pose relative to the door handle, while moving the dishwasher with a joint goal for its hinge.
This indirectly causes the whole robot to move its gripper in an opening motion, as the dishwasher moves in accordance to its joint goal.

**Exercise: try different joint goals**

In [ ]:
reset_giskard()

# load the kitchen environment
kitchen_pose = PoseStamped()
kitchen_pose.header.frame_id = 'map'
kitchen_pose.pose.orientation.w = 1
giskard.world.add_urdf(name='iai_kitchen',
                       urdf=rospy.get_param('kitchen_description'),
                       pose=kitchen_pose)

# Define poses and groups
p = PoseStamped()
p.header.frame_id = 'map'
p.pose.orientation.w = 1
p.pose.position.x = 0.5
p.pose.position.y = 0.2

hand = tool_frame

goal_angle = np.pi / 4
handle_frame_id = 'sink_area_dish_washer_door_handle'
handle_name = 'sink_area_dish_washer_door_handle'
giskard.world.register_group(new_group_name='dishwasher', root_link_name='sink_area_dish_washer_main',
                             root_link_group_name='iai_kitchen')
giskard.world.register_group(new_group_name='handle', root_link_name=handle_name,
                             root_link_group_name='iai_kitchen')

# grasp the dishwasher handle
bar_axis = Vector3Stamped()
bar_axis.header.frame_id = handle_frame_id
bar_axis.vector.y = 1

bar_center = PointStamped()
bar_center.header.frame_id = handle_frame_id

tip_grasp_axis = Vector3Stamped()
tip_grasp_axis.header.frame_id = hand
tip_grasp_axis.vector.z = 1

giskard.motion_goals.add_grasp_bar(root_link='map',
                                   tip_link=hand,
                                   tip_grasp_axis=tip_grasp_axis,
                                   bar_center=bar_center,
                                   bar_axis=bar_axis,
                                   bar_length=.3)

x_gripper = Vector3Stamped()
x_gripper.header.frame_id = hand
x_gripper.vector.x = 1

x_goal = Vector3Stamped()
x_goal.header.frame_id = handle_frame_id
x_goal.vector.x = -1
giskard.motion_goals.add_align_planes(tip_link=hand,
                                      root_link='map',
                                      tip_normal=x_gripper,
                                      goal_normal=x_goal)

if ROBOT == 'stretch':
    x_base = Vector3Stamped()
    x_base.header.frame_id = 'base_link'
    x_base.vector.x = 1
    x_goal = Vector3Stamped()
    x_goal.header.frame_id = 'map'
    x_goal.vector.y = 1
    giskard.motion_goals.add_align_planes(tip_link='base_link',
                                          root_link='map',
                                          tip_normal=x_base,
                                          goal_normal=x_goal,
                                          name='align_base')

giskard.add_default_end_motion_conditions()
giskard.execute()

# open the dishwasher
giskard.motion_goals.add_open_container(tip_link=hand,
                                        environment_link=handle_name,
                                        goal_joint_state=goal_angle)
giskard.motion_goals.allow_all_collisions()
giskard.add_default_end_motion_conditions()
giskard.execute()

# close the dishwasher
giskard.motion_goals.add_open_container(tip_link=hand,
                                        environment_link=handle_name,
                                        goal_joint_state=0)
giskard.motion_goals.allow_all_collisions()
giskard.add_default_end_motion_conditions()
assert giskard.execute().error.code == GiskardError.SUCCESS

# Section 3: Monitors and Giskard Task Graphs
**Tutorial 3.1: Simple sequencing of to cartesian pose goals in one controller**

Until now, we have seen different motion goals and monitors to observe the success of the specified Motion Goals, i.e. constraints.
Another way to specify a motion with giskard is to utilize monitors and the start-, to_hold- and end_condition's each motion goal and monitor offer.
Doing this, a chain, or more specifically a graph (task graph in Giskard terms), of motion goals is build up.
The task graph will be executed as one continuous controller (no need to parse and recompile a new one for each goal).
After execution a picture of the task graph and a Gant diagram visualizing the execution order of goals and monitors can be inspected.
Next we will see simple example to execute two cartesian poses after each other.

In [32]:
reset_giskard()

pose1 = PoseStamped()
pose1.header.frame_id = 'map'
pose1.pose.position.x = 1
pose1.pose.orientation.w = 1

pose2 = PoseStamped()
pose2.header.frame_id = base_link
pose2.pose.position.y = 1
pose2.pose.orientation.w = 1

root_link = 'map'
tip_link = base_link

monitor1 = giskard.monitors.add_cartesian_pose(name='pose1',
                                               root_link=root_link,
                                               tip_link=tip_link,
                                               goal_pose=pose1)

cart_goal_reached = giskard.monitors.add_cartesian_pose(name='pose2',
                                                        root_link=root_link,
                                                        tip_link=tip_link,
                                                        goal_pose=pose2,
                                                        absolute=True,
                                                        start_condition=monitor1,
                                                        position_threshold=0.05,
                                                        orientation_threshold=0.05)
local_min_reached = giskard.monitors.add_local_minimum_reached()

giskard.motion_goals.add_cartesian_pose(goal_pose=pose1,
                                        name='g1',
                                        root_link=root_link,
                                        tip_link=tip_link,
                                        end_condition=monitor1)
if ROBOT == 'pr2':
    giskard.motion_goals.add_cartesian_pose(goal_pose=pose2,
                                            name='g2',
                                            root_link=root_link,
                                            tip_link=tip_link,
                                            absolute=True,
                                            start_condition=monitor1,
                                            end_condition=f'{cart_goal_reached} and {local_min_reached}')
elif ROBOT == 'stretch':
    giskard.motion_goals.add_diff_drive_base(goal_pose=pose2,
                                             tip_link=base_link,
                                             name='g2',
                                             root_link='map',
                                             start_condition=monitor1,
                                             end_condition=f'{cart_goal_reached} and {local_min_reached}')
giskard.motion_goals.allow_all_collisions()
giskard.monitors.add_end_motion(start_condition=f'{cart_goal_reached} and {local_min_reached}')
giskard.monitors.add_max_trajectory_length(50)
assert giskard.execute().error.code == GiskardError.SUCCESS

**Visualize the Task Graph**

In [ ]:
visualize_last_task_graph()

**Visualize the Gant diagramm**

In [ ]:
visualize_last_gant_diagram()

**Tutorial 3.2: Complex Sequencing**

This can be utilized to build arbitrary complex task graphs.

**Exercise a): Read the code and compare your expectations with the execution, the printed out task graph and the Gantt diagramm.**

**Exercise b): Come up with your own sequences using everything we have looked at so far**

In [ ]:
reset_giskard()
if ROBOT != 'pr2':
    raise Exception('This Tutorial only works with the PR2.')

# %% Define goals for later
right_arm_goal = {'r_shoulder_pan_joint': -1.7125,
                  'r_shoulder_lift_joint': -0.25672,
                  'r_upper_arm_roll_joint': -1.46335,
                  'r_elbow_flex_joint': -2.12,
                  'r_forearm_roll_joint': 1.76632,
                  'r_wrist_flex_joint': -0.10001,
                  'r_wrist_roll_joint': 0.05106}

left_arm_goal = {'l_shoulder_pan_joint': 1.9652,
                 'l_shoulder_lift_joint': - 0.26499,
                 'l_upper_arm_roll_joint': 1.3837,
                 'l_elbow_flex_joint': -2.12,
                 'l_forearm_roll_joint': 16.99,
                 'l_wrist_flex_joint': - 0.10001,
                 'l_wrist_roll_joint': 0}

base_goal = PoseStamped()
base_goal.header.frame_id = 'map'
base_goal.pose.position.x = 2
base_goal.pose.orientation.w = 1

# %% Monitors observe something and turn to True, if the condition is met. They don't cause any motions.
# All monitor related operations are grouped under giskard_wrapper.monitors.
# Let's define a few
# This one turns True when the length of the current trajectory % mod = 0
alternator = giskard.monitors.add_alternator(mod=2)
# This one sleeps and then turns True
sleep1 = giskard.monitors.add_sleep(1, name='sleep1')
# This prints a message and then turns True.
# With start_condition you can define which monitors need to be True in order for this one to become active
print1 = giskard.monitors.add_print(message=f'{sleep1} done', start_condition=sleep1)
# You can also write logical expressions using "and", "or" and "not" to combine multiple monitors
sleep2 = giskard.monitors.add_sleep(1.5, name='sleep2', start_condition=f'{print1} or not {sleep1}')

# %% Now Let's define some motion goals.
# We want to reach two joint goals, so we first define monitors for checking that end condition.
right_monitor = giskard.monitors.add_joint_position(goal_state=right_arm_goal,
                                                    name='right pose reached',
                                                    start_condition=sleep1)
# You can use add_motion_goal to add any monitor implemented in giskardpy.monitor.
# All remaining parameters are forwarded to the __init__ function of that class.
# All specialized add_ functions are just wrappers for add_monitor.
left_monitor = giskard.monitors.add_monitor(monitor_class=JointGoalReached.__name__,
                                            goal_state=left_arm_goal,
                                            name='left pose reached',
                                            start_condition=sleep1,
                                            threshold=0.01)

# We set two separate motion goals for the joints of the left and right arm.
# All motion goal related operations are groups under giskard.motion_goals.
# The one for the right arm starts when the sleep2 monitor is done and ends, when the right_monitor is done,
# meaning it continues until the joint goal was reached.
giskard.motion_goals.add_joint_position(goal_state=right_arm_goal,
                                        name='right pose',
                                        start_condition=sleep2,
                                        end_condition=right_monitor)
# You can use add_motion_goal to add any motion goal implemented in giskardpy.goals.
# All remaining parameters are forwarded to the __init__ function of that class.
giskard.motion_goals.add_motion_goal(motion_goal_class=JointPositionList.__name__,
                                     goal_state=left_arm_goal,
                                     name='left pose',
                                     end_condition=left_monitor)

# %% Now let's define a goal for the base, 2m in front of it.
# First we define a monitor which checks if that pose was reached.
base_monitor = giskard.monitors.add_cartesian_pose(root_link='map',
                                                   tip_link='base_footprint',
                                                   goal_pose=base_goal)

# and then we define a motion goal for it.
# The hold_condition causes the motion goal to hold as long as the condition is True.
# In this case, the cart pose is halted if time % 2 == 1 and active if time % 2 == 0.
giskard.motion_goals.add_cartesian_pose(root_link='map',
                                        tip_link='base_footprint',
                                        goal_pose=base_goal,
                                        hold_condition=f'not {alternator}',
                                        end_condition=base_monitor)

# %% Define when the motion should end.
# Usually you'd use the local minimum reached monitor for this.
local_min = giskard.monitors.add_local_minimum_reached()

# Giskard will only end the motion generation and return Success, if an end monitor becomes True.
# We do this by defining one that gets triggered, when a local minimum was reached, sleep2 is done and the motion goals
# were reached.
giskard.monitors.add_end_motion(start_condition=' and '.join([local_min,
                                                              sleep2,
                                                              right_monitor,
                                                              left_monitor,
                                                              base_monitor]))
# It's good to also add a cancel condition in case something went wrong and the end motion monitor is unable to become
# True. Currently, the only predefined specialized cancel monitor is max trajectory length.
# Alternative you can use monitor.add_cancel_motion similar to end_motion.
giskard.monitors.add_max_trajectory_length(120)
# Lastly we allow all collisions
giskard.motion_goals.allow_all_collisions()
# And execute the goal.
assert giskard.execute().error.code == GiskardError.SUCCESS

In [ ]:
visualize_last_task_graph()

In [ ]:
visualize_last_gant_diagram()

# Section 4: Integration in the CRAM Cognitive Architecture
In the CRAM cognitive architecture a task is composed of motion phases that are aligned with the Flanagan model.
These are implemented as motion designators in PyCram. It utilizes the motion designator structure and related knowledge from various knowledge bases and services to infer motion parameters that are used to specify the resulting physical motion using Giskard.


<img src="generalized-motion-plan-alt.png" width="1100"/>

**Tutorial 4.1: CRAM Motion Sequence**
Here we will show how all the above motion phases can be programmed as one giskard task graph. In the PyCram implementation and will first infer the motion paramaters and then call the respective Giskard interface functions on its own. For this example the parameters are hard coded.

In [36]:
reset_giskard()
# define an object
obj_pose = PoseStamped()
obj_pose.header.frame_id = 'map'
obj_pose.pose.orientation.w = 1
obj_pose.pose.position = Point(1.2, -0.2, 0.9)
giskard.world.add_box(name='box', size=(0.04, 0.1, 0.2), pose=obj_pose, parent_link='map')

# define motion parameters. None params will be inferred later
object_param = 'box'
arm_param = tool_frame
reach_pose_param = None
grasp_pose_param = None
force_param = None
lift_pose_param = None
place_location_pose_param = None
reach_pose2_param = None
destination_pose_param = None
retreat_pose_param = None

# the reach and grasp poses can be inferred as poses relative to the object
grasp_pose_param = PoseStamped()
grasp_pose_param.header.frame_id = object_param
grasp_pose_param.pose.orientation = Quaternion(*quaternion_from_matrix([[0, 1, 0, 0],
                                                                        [-1, 0, 0, 0],
                                                                        [0, 0, 1, 0],
                                                                        [0, 0, 0, 1]]))
grasp_pose_param.pose.position = Point(0, 0.05, 0)

reach_pose_param = deepcopy(grasp_pose_param)
reach_pose_param.pose.position.y += 0.1

# as we use a simulated robot we replace the force_param with a gripper joint goal
gripper_joint_goal_open_param = {gripper_joint: gripper_joint_open}
gripper_joint_goal_close_param = {gripper_joint: gripper_joint_close}

# the lift pose can be calculated as some pose above the grasp pose
lift_pose_param = deepcopy(grasp_pose_param)
lift_pose_param.pose.position.z += 0.1

# for the place_location_pose_param we assume that a region of interest exist in some x/y coordinates. Therefore, we want to move the base there.
place_location_pose_param = PoseStamped()
place_location_pose_param.header.frame_id = 'map'
place_location_pose_param.pose.orientation.w = 1
place_location_pose_param.pose.position = Point(2.5, 2, 0)

# for the destination and reach_pose2 we define a destination pose close to our region on interest and a reach pose above it to simulate placing the box onto something. For the retreat pose when then choose a pose that allows the gripper to move away from the object without collision.
destination_pose_param = PoseStamped()
destination_pose_param.header.frame_id = 'map'
destination_pose_param.pose.orientation.w = 1
destination_pose_param.pose.position = Point(3.5, 2, 0.8)

reach_pose2_param = deepcopy(destination_pose_param)
reach_pose2_param.pose.position.z += 0.1

retreat_pose_param = deepcopy(destination_pose_param)
retreat_pose_param.pose.position.x -= 0.1

# Now the sequence of cartesian and joint goals is created. We have to split that into three sequences to attach and detach the object from the gripper.
# Through attaching the object becomes a part of the kinematic tree of the robot.

# first we create the monitors and motion goals for the first phase until we are in contact with the object
open_gripper_monitor = giskard.monitors.add_joint_position(goal_state=gripper_joint_goal_open_param, name='openGripper')
reach_monitor = giskard.monitors.add_cartesian_pose(root_link='map', tip_link=arm_param, goal_pose=reach_pose_param,
                                                    start_condition=open_gripper_monitor, name='reach')
grasp_monitor = giskard.monitors.add_cartesian_pose(root_link='map', tip_link=arm_param, goal_pose=grasp_pose_param,
                                                    start_condition=reach_monitor, name='grasp')
close_gripper_monitor = giskard.monitors.add_joint_position(goal_state=gripper_joint_goal_close_param,
                                                            start_condition=grasp_monitor, name='closeGripper')

giskard.motion_goals.add_joint_position(goal_state=gripper_joint_goal_open_param, end_condition=open_gripper_monitor, name='open')
giskard.motion_goals.add_cartesian_pose(goal_pose=reach_pose_param, tip_link=arm_param, root_link='map',
                                        start_condition=open_gripper_monitor, end_condition=reach_monitor, name='reach')
giskard.motion_goals.add_cartesian_pose(goal_pose=grasp_pose_param, tip_link=arm_param, root_link='map',
                                        start_condition=reach_monitor, end_condition=grasp_monitor, name='grasp')
giskard.motion_goals.add_joint_position(goal_state=gripper_joint_goal_close_param, start_condition=grasp_monitor,
                                        end_condition=close_gripper_monitor, name='close')
giskard.monitors.add_end_motion(close_gripper_monitor)
giskard.motion_goals.allow_all_collisions()
giskard.monitors.add_max_trajectory_length(50)
giskard.execute()

# then we attach the box to the arm and continue the sequence until we have reached the destination
lift_monitor = giskard.monitors.add_cartesian_pose(root_link='map', tip_link=arm_param, goal_pose=lift_pose_param, name='lift')
loc_pos_monitor = giskard.monitors.add_cartesian_pose(root_link='map', tip_link=base_link, orientation_threshold=0.05, position_threshold=0.05,
                                                      goal_pose=place_location_pose_param, start_condition=lift_monitor, name='location')
reach2_monitor = giskard.monitors.add_cartesian_pose(root_link='map', tip_link=arm_param, goal_pose=reach_pose2_param,
                                                     start_condition=loc_pos_monitor, name='reach2')
dest_monitor = giskard.monitors.add_cartesian_pose(root_link='map', tip_link=arm_param, goal_pose=destination_pose_param,
                                                   start_condition=reach2_monitor, name='dest')
open_gripper2_monitor = giskard.monitors.add_joint_position(goal_state=gripper_joint_goal_open_param, start_condition=dest_monitor, name='openGripper2')

giskard.world.update_parent_link_of_group(object_param, arm_param)
giskard.motion_goals.add_cartesian_pose(goal_pose=lift_pose_param, tip_link=arm_param, root_link='map',
                                        end_condition=lift_monitor, name='lift')
if ROBOT == 'pr2':
    giskard.motion_goals.add_cartesian_pose(goal_pose=place_location_pose_param, tip_link=base_link, root_link='map',
                                            start_condition=lift_monitor, end_condition=loc_pos_monitor, name='location')
elif ROBOT == 'stretch':
    giskard.motion_goals.add_diff_drive_base(goal_pose=place_location_pose_param, tip_link=base_link, name='location', root_link='map',
                                             start_condition=lift_monitor, end_condition=loc_pos_monitor)
giskard.motion_goals.add_cartesian_pose(goal_pose=reach_pose2_param, tip_link=arm_param, root_link='map',
                                        start_condition=loc_pos_monitor, end_condition=reach2_monitor, name='reach2')
giskard.motion_goals.add_cartesian_pose(goal_pose=destination_pose_param, tip_link=arm_param, root_link='map',
                                        start_condition=reach2_monitor, end_condition=dest_monitor, name='dest')
giskard.motion_goals.add_joint_position(goal_state=gripper_joint_goal_open_param, start_condition=dest_monitor, end_condition=open_gripper2_monitor, name='open2')
giskard.monitors.add_end_motion(open_gripper2_monitor)
giskard.motion_goals.allow_all_collisions()
giskard.monitors.add_max_trajectory_length(50)
giskard.execute()

# then we detach the object and retreat away from it
giskard.world.update_parent_link_of_group(object_param, 'map')
retreat_monitor = giskard.monitors.add_cartesian_pose(root_link='map', tip_link=arm_param, goal_pose=retreat_pose_param, name='retreat')
giskard.motion_goals.add_cartesian_pose(goal_pose=retreat_pose_param, tip_link=arm_param, root_link='map',
                                        end_condition=retreat_monitor, name='retreat')
giskard.monitors.add_end_motion(retreat_monitor)
giskard.motion_goals.allow_all_collisions()
giskard.monitors.add_max_trajectory_length(50)
assert giskard.execute().error.code == GiskardError.SUCCESS